<a href="https://colab.research.google.com/github/sanghyun-ai/ktcloud_genai/blob/main/203_LLM_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **텍스트 분류(Text Classification)**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



In [2]:
# %%capture : 코드 셀의 출력을 숨겨
%%capture
!pip install datasets

In [3]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()



---



## **텍스트 분류(Text Classification) 이해**

- **분류** :  가장 일반적인 자연어 처리 작업
- **분류 작업의 목표**:
    - 모델을 훈련하여 입력 테스트에 레이블(label) or 클래스(class)를 할당하는 것
- **텍스트 분류**:
    - **감성 분석**(sentiment analysis)
    - **의도 감지**(intent detection)
    - **엔티티 추출**(entity extraction)
    - **언어 감지** 등 광범위한 애플리케이션에서 사용됨
- **표현 언어 모델과 생성 언어 모델이 분류 작업에 많은 영향을 끼치고 있음**
    - 표현(Representation) 언어 모델
        - 세상의 모든 글을 읽고 깊이 이해하는 독해 전문가
    - 생성(Generative) 언어 모델
        - 하나의 주제를 바탕으로 유창하게 새로운 글을 쓰는 작문 전문가

| 구분	| 표현 언어 모델 (Representation Model)	| 생성 언어 모델 (Generative Model)	| 인코더-디코더 모델 (Seq2Seq Model) |
|--- | ---|---|---|
| 목표	|  문장의 의미 이해 및 표현 (Understanding)	|  다음 단어 예측을 통한 새로운 문장 생성 (Generation)	|  입력 문장 이해 후, 새로운 형태의 문장 생성|
| 핵심 아이디어	|  양방향(Bidirectional) 문맥 파악	|  단방향(Unidirectional) 문맥 기반 예측	|  **이해**(인코더)와 **생성**(디코더)의 결합	|
| 대표 모델	|  **BERT**, RoBERTa, ALBERT, ELECTRA	|  **GPT** 계열, LLaMA, Claude, PaLM	|  **T5**, BART, (초기) Transformer	|
| 주요 구조	|  트랜스포머 인코더 (Encoder)	|  트랜스포머 디코더 (Decoder)	|  트랜스포머 인코더 + 디코더	|
| 학습 방식	|  MLM (Masked Language Model): 문장의 일부 단어를 가리고 맞추는 방식| 	CLM (Causal Language Model): 이전 단어들로 다음 단어를 예측하는 방식	|  두 가지 방식을 혼합하여 사용	|
| 비유	|  🧑‍🏫 독해 전문가, 맥락 탐정	|  ✍️ 창의적인 작가, 이야기꾼	|  🌐 능숙한 통역가, 요약 전문가	|
| 잘하는 작업	"|  - 분류: 감성 분석, 뉴스 카테고리 분류<br>- 개체명 인식(NER): 인물, 장소 찾기<br>- 질의응답: 본문에서 정답 찾기<br>- **작업에 특화된 모델과 임베딩 생성 모델로 사용**| - 창작: 챗봇, 소설/기사 작성, 이메일 초안<br>- 요약: 긴 글을 짧게 요약<br>- 코드 생성: 기능에 맞는 코드 작성"	"| - 번역: 한국어 → 영어<br>- 요약: (특히) 입력과 출력의 형태가 다른 요약"	|


- **(텍스트 분류 작업을 위한) 허깅페이스에서의 모델 사용 방법(task명)**
    - **1.텍스트 분류 모델 사용**(text-classification) :
        - https://huggingface.co/models?pipeline_tag=text-classification
    - **2.임베딩 생성 모델 사용**(feature-extraction) :
        - https://huggingface.co/models?pipeline_tag=feature-extraction


---



### **분류에 사용할 데이터셋**(영화 리뷰 데이터셋)

- 허깅페이스 데이터셋 : https://huggingface.co/datasets
- 로튼 토마토 (Rotten Tomatoes) 데이터셋:
    - https://www.rottentomatoes.com/
    - 영화와 TV 프로그램에 대한 리뷰를 모아놓은(review-aggregation) 미국의 웹사이트
    - 자연어 처리(NLP) 연구에 매우 중요하게 사용
    - 데이터 구성
        - Review (리뷰 텍스트)
        - Label (감성 라벨): 긍정(1,positive), 부정(0,negative)

In [4]:
from datasets import load_dataset

# 데이터를 로드합니다.
data = load_dataset("rotten_tomatoes")
data
# 훈련, 검증, 테스트 데이터셋으로 구분됨
# - 검증 데이터셋 : 모델을 훈련할 때 파라미터 튜닝할 경우 사용할 수 있음

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [5]:
print(data["train"].shape)
data["train"][0, -1]  # 첫 행의 마지막 열 값

(8530, 2)


{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

### **표현 모델로 텍스트 분류하기(BERT 계열)**

- BERT 계열 모델은 **작업(task)에 특화된 모델과 임베딩 생성 모델로 사용됨**
- BERT 변종 베이스 모델 (자세한 내용은 각 논문 찾아보기)
    - **BERT 베이스 모델**(uncased) : https://huggingface.co/google-bert/bert-base-uncased
    - **RoBERTa 베이스 모델** : https://huggingface.co/FacebookAI/roberta-base
    - **DistilBERT 베이스 모델** : https://huggingface.co/distilbert/distilbert-base-uncased
    - **DeBERTa 베이스 모델** : https://huggingface.co/microsoft/deberta-base
    - **bert-tiny** : https://huggingface.co/prajjwal1/bert-tiny
    - **ALBERT 베이스 V2** : https://huggingface.co/albert/albert-base-v2
- 파인튜닝 모델 예:
    - **Twitter-RoBERTa-base 모델**:  감성 분석을 위해 트윗 데이터에서 미세 튜닝한 RoBERTa 모델
        - https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest



### **예제: 나만의 영화 평론 분석기**
- [참고] 103-LLM-언어 AI 맛보기.ipynb

In [6]:
# Hugging Face의 transformers 라이브러리 활용
from transformers import pipeline

# 감성 분석 파이프라인 로드 (sentiment-analysis는 "작업 유형"을 지정하는 것)
sentiment_analyzer = pipeline("sentiment-analysis")  # Task : sentiment-analysis

# review = "이 영화는 정말 시간 가는 줄 모르고 봤어요. 배우들 연기가 최고!"
review = "어떻게 감독이 된 건지 의문인 영화"
result = sentiment_analyzer(review)

print('\n✅ 감성 분석(Sentiment Analysis) 결과')
print(f"리뷰: {review}")
print(f"분석 결과: {result}") # [{'label': 'POSITIVE', 'score': 0.99...}]

# 핵심 구문 추출은 별도의 모델이나 기법이 필요
# 예시: "배우들 연기", "시간 가는 줄" 등을 추출

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu



✅ 감성 분석(Sentiment Analysis) 결과
리뷰: 어떻게 감독이 된 건지 의문인 영화
분석 결과: [{'label': 'POSITIVE', 'score': 0.7819299697875977}]


In [7]:
from transformers import pipeline
import pandas as pd

# 감정 분석 파이프라인 초기화
#   model=<업로더이름>/<모델이름>
#       nlptown → 모델을 공개한 Hugging Face 사용자(혹은 조직) 계정명
#       bert-base-multilingual-uncased-sentiment → 모델의 이름
#           BERT 기반
#           다국어 지원
#           대소문자 구분하지 않음 (uncased)
#           감성 분석용으로 학습된 모델
sentiment_analyzer = pipeline("sentiment-analysis",
                             model="nlptown/bert-base-multilingual-uncased-sentiment" )


# 샘플 리뷰 데이터
reviews = [
    "이 제품 정말 좋아요! 추천합니다.",
    "배송이 너무 늦었어요. 실망스럽네요.",
    "가격 대비 괜찮은 것 같습니다."
]

# 감정 분석 수행
results = []
for review in reviews:
    result = sentiment_analyzer(review)
    results.append({
        'text': review,
        'sentiment': result[0]['label'],
        'confidence': result[0]['score']
    })

# 결과 출력
df = pd.DataFrame(results)

print('\n✅ 감성 분석(Sentiment Analysis) 결과')
print(df)

Device set to use cpu



✅ 감성 분석(Sentiment Analysis) 결과
                   text sentiment  confidence
0   이 제품 정말 좋아요! 추천합니다.   5 stars    0.718316
1  배송이 너무 늦었어요. 실망스럽네요.    1 star    0.425818
2     가격 대비 괜찮은 것 같습니다.   3 stars    0.556177




---



## **1. 작업에 특화된 모델 사용하기**



### **예제: 감성분석을 위한 사전학습모델을 사용**
- 💡 아래 예제는 감성분석을 위한 사전학습모델을 사용합니다.

- **데이터셋 준비하기**
    - 앞에서 실행한 내용

In [8]:
# 허깅페이스 데이터셋 사용
from datasets import load_dataset

# rotten_tomatoes에서 데이터를 로드합니다.
data = load_dataset("rotten_tomatoes")
data
# 훈련, 검증, 테스트 데이터셋으로 구분됨
# - 검증 데이터셋 : 모델을 훈련할 때 파라미터 튜닝할 경우 사용할 수 있음

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

**1. 모델 로드하기**
    - 훈련 데이터 없는 단어를 만나더라도 토큰을 결합하여 표현을 생성할 수 있다.

In [9]:
from transformers import pipeline

# 허깅 페이스 모델 경로
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# 파이프라인으로 모델을 로드합니다.
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

**2. 테스트 세트로 모델 실행**

In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# 추론을 실행합니다.
y_pred = []   # 예측 결과
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    negative_score = output[0]["score"]
    positive_score = output[2]["score"]
    assignment = np.argmax([negative_score, positive_score])
    y_pred.append(assignment)
# output: [{'label': 'negative', 'score': 0.00516123790293932}, {'label': 'neutral', 'score': 0.040233541280031204}, {'label': 'positive', 'score': 0.9546052813529968}]

**3. 모델 평가하기**

In [ ]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """분류 리포트를 만들어 출력합니다."""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

# 모델 평가하기
evaluate_performance(data["test"]["label"], y_pred)

# 영화 리뷰에서 훈련하지 않은 모델이지만(사전훈련모델) F1-score가 0.8이면 괜찮은 성능

- **혼동행렬(Confusion Matrix)**

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 혼동 행렬 계산
cm = confusion_matrix(data["test"]["label"], y_pred)

# 2. 클래스 이름 정의
class_names = ["Negative Review", "Positive Review"]

# 3. 혼동 행렬을 히트맵으로 시각화
plt.figure(figsize=(6, 4)) # 그래프 크기 설정
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label') # x축 라벨
plt.ylabel('True Label')      # y축 라벨
plt.title('Confusion Matrix') # 그래프 제목
plt.show()

### **[실습] 다른 모델 사용해서 Accuracy 성능 높여보기**
- **사용 모델** : distilbert/distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
#------------------
# 1.모델 로드하기
#------------------
from transformers import pipeline

# 허깅 페이스 모델 경로
model_path = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

# 파이프라인으로 모델을 로드합니다.
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

#------------------
# 2.테스트 세트로 모델 실행
#------------------
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# 추론을 실행합니다.
y_pred = []   # 예측 결과
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    negative_score = output[0]["score"]
    positive_score = output[1]["score"]
    assignment = np.argmax([negative_score, positive_score])
    y_pred.append(assignment)
# output: [{'label': 'NEGATIVE', 'score': 0.00018543035548646003}, {'label': 'POSITIVE', 'score': 0.9998145699501038}]

#------------------
# 3.모델 평가하기
#------------------
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """분류 리포트를 만들어 출력합니다."""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

# 모델 평가하기
evaluate_performance(data["test"]["label"], y_pred)




---



## **2. 임베딩을 활용하여 분류 작업 수행하기**

### **임베딩을 생성하는 모델 선택하는 방법**

- **(일반적 성능 확인)MTEB 리더보드를 우선 찾아본다**.
    - https://huggingface.co/spaces/mteb/leaderboard
    - MTEB (Massive Text Embedding Benchmark)
        - 허깅페이스 연구팀에서 제공하는 다양한 자연어 처리(NLP) 과제에서 **임베딩 모델(embedding model)의 품질**을 객관적으로 평가하기 위한 벤치마크
        - 58개 데이셋과 112개의 언어를 포괄하는 8개의 임베딩 평가 작업으로 구성됨
        - **2024년 이후 최신 임베딩 모델의 비교 평가 지표로 MTEB 사용 확산**
        - 영어 중심이지만, Korean 포함 다국어(Multilingual MTEB)도 지원
        - 성능 뿐만 아니라 추론 속도 등 다양한 항목 고려해야됨

- **(자신의 목적에 맞게: *텍스트 분류*) 임베딩 모델(BERT 계열) 사용**
    - sentence-transformers/all-mpnet-base-v2** 사용
    - https://huggingface.co/sentence-transformers/all-mpnet-base-v2
    - 작지만 성능 좋다.



- **MTEB로 평가되는 대표 과제(Task) 유형**

|카테고리|설명|예시|
|---|---|---|
|Classification|텍스트 분류|뉴스 기사 주제 분류|
|Clustering|문장 또는 문서 군집화|유사 뉴스 기사 묶기|
|Pair Classification|문장 쌍 관계 분류|문장1과 문장2가 같은 의미인가?|
|STS (Semantic Textual Similarity)|문장 의미 유사도 측정|“나는 학교에 간다” ↔ “나는 학교로 갔다”|
|Retrieval|검색 및 정보 검색|질문에 맞는 문서 찾기|
|Reranking|검색 결과 재정렬|검색 결과 중 더 관련성 높은 순서로 정렬|
|Summarization / Bitext Mining 등|"요약, 번역 등 다양"|"문서요약, 병렬문장 탐색 등"|

- **MTEB 사용 예**

In [ ]:
!pip install mteb

In [ ]:
from mteb import MTEB
from sentence_transformers import SentenceTransformer

# 1. 평가할 모델 로드
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 2. 벤치마크 선택 (예: 영어 STS)
evaluation = MTEB(tasks=["STSBenchmark"])

# 3. 평가 실행
evaluation.run(model, output_folder="results")

# 결과는 ./results 폴더에 JSON 형식으로 저장됨


### **예제: 지도 학습 분류**
- **작업 방법**
    1. **임베딩 모델을 사용해 특성을 생성**(임베딩 생성)
    2. **특성을 분류기에 주입**(ex: 로지스틱 회귀로 분류)
- **이점** :
    - 비용이 많이 들 수 있는 임베딩 모델을 미세 튜닝할 필요 없다.
    - CPU에서 로지스틱 회귀와 같은 분류기를 훈련하면 된다.
- **사용 임베딩 모델**
    - sentence-transformers/all-mpnet-base-v2

**1. 임베딩 모델을 사용해 특성을 생성**(임베딩 생성)

In [ ]:
from sentence_transformers import SentenceTransformer

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 텍스트를 임베딩으로 변환합니다.
train_embeddings = model.encode(list(data["train"]["text"]), show_progress_bar=True)
test_embeddings = model.encode(list(data["test"]["text"]), show_progress_bar=True)

In [ ]:
train_embeddings.shape

**2. 특성을 분류기에 주입**(ex: 로지스틱 회귀로 분류)

In [ ]:
from sklearn.linear_model import LogisticRegression

# 훈련 세트의 임베딩으로 로지스틱 회귀 모델을 훈련합니다.
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

**3. 모델 평가**

In [ ]:
# 테스트 세트 임베딩에 대해 예측을 수행합니다.
y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

- 💡**팁!**  

분류기를 사용하지 않는다면 어떻게 할 수 있을까요? 분류기 대신 클래스별 임베딩을 평균하고 코사인 유사도를 적용하여 문서와 가장 잘 맞는 클래스를 예측할 수 있습니다.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

# 타깃 레이블에 대한 문서의 임베딩을 모두 평균하여 타깃 임베딩을 만듭니다.
df = pd.DataFrame(np.hstack([train_embeddings, np.array(data["train"]["label"]).reshape(-1, 1)]))
averaged_target_embeddings = df.groupby(768).mean().values

# 테스트 임베딩과 가장 가까운 타깃 임베딩을 찾습니다.
sim_matrix = cosine_similarity(test_embeddings, averaged_target_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

# 모델을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)

### **예제: 데이터에 레이블이 없는 경우**

- **제로샷 분류** (ZeroShotClassification) 적용
- **원리**: 라벨을 “설명 문장(hypothesis)”으로 바꿔서, 자연어 추론(NLI, Natural Language Inference) 문제로 변환 후 모델이 참/거짓을 판단하도록 함.
- **예**:
    - 문장: "이 영화는 정말 재밌었다"
    - 라벨 후보: ["긍정", "부정"]
    - **변환** → "이 문장은 긍정을 표현한다." (참/거짓 예측)

In [ ]:
# 레이블의 임베딩을 만듭니다.
label_embeddings = model.encode(["A negative review",  "A positive review"])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 각 문서와 가장 잘 맞는 레이블을 찾습니다.
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

### **[실습] 제로샷 레이블을 변경하여 성능 향상시키기**

- 레이블 설명을 다르게 설정하여 모델 평가하기
    - A negative review --> **A very negative movie review**
    - A positive review --> **A very positive movie review** 로 변경

In [ ]:
label_embeddings = model.encode(["A very negative movie review",
                                 "A very positive movie review"])

sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

evaluate_performance(data["test"]["label"], y_pred)

# 설명에 있는 레이블 내용(단어)를 통해 임베딩은 영화 리뷰임을 감지하고 좀 더 극단적인 레이블에 초점을 맞춘다.



---



## **3. 생성 모델로 텍스트 분류하기**

- **GPT 모델**과 같은 **생성 언어 모델**은 아무 맥락 없이 리뷰를 제공하면 모델이 무엇을 해야할지 알지 못합니다.
- **모델이 맥락을 이해하도록 돕기 위해** 모델에 제공되는 명령(or **프롬프트**)을 통해 이루어집니다.
- 원하는 출력을 얻도록 **프롬프트를 반복적으로 개선**하는 것을 **프롬프트 엔지니어링**(prompt engineering)이라고 부룹니다.

### **예제: T5 모델 사용하기**
- **T5: Text-To-Text Transfer Transformer**
- 시퀀스-투-시퀀스 모델(Seq2Seq)
- 원본 트랜스포머와 비슷하게 12개의 디코더+12개의 인코더를 쌓아서 구성됨
- 마스크드 언어 모델링을 사용해 사전 훈렴됨
- **각각의 작업을 시퀀스-투-시퀀스 작업으로 변환하고 동시에 훈련한다**.--> **다양한 작업에 대해 모델을 훈련할 수 있다**.
- 이러한 미세 튜닝방법으로 GPT 모델에 가까운 수준으로 명령을 따른다. https://arxiv.org/abs/2210.11416
- 이 작업으로 만들어진 모델이 **Flan-T5**다.
- **다양한 Flan-T5 모델**
    - google/flan-t5-base : https://huggingface.co/google/flan-t5-base
    - google/flan-t5-small : https://huggingface.co/google/flan-t5-small
    - google/flan-t5-large : https://huggingface.co/google/flan-t5-large


**1. Flan-T5 모델 로드하기**

In [ ]:
# 모델을 로드합니다.
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device="cuda:0"
)

**2. 프롬프트 추가하기**

In [ ]:
# 프롬프트를 추가합니다.
prompt = "Is the following sentence positive or negative? "
data = data.map(lambda example: {"t5": prompt + example['text']})
data

**3. 테스트 데이터 실행**

In [ ]:
# 추론을 실행합니다.
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):
    text = output[0]["generated_text"]
    y_pred.append(0 if text == "negative" else 1)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

### **예제: ChatGPT로 분류하기**
- **클로즈드 모델**(closed model)
- **훈련과정 소개** : https://openai.com/ko-KR/index/chatgpt/
- **모델 생성 과정**
    1. 입력 프롬프트에 대해 기대하는 출력을 수동으로 만든다.
        - **모범 답안 학습하기**(지시 데이터:instruction data)
        - --> 모델의 첫번째 버전 생성
    2. 만들어진 모델을 사용해 여러 출력을 생성하고 가장 나쁜 에서 가장 좋은 것까지 수동으로 순위를 매김
        - **선호도 데이터 사용**
        - --> 최종 모델 만든다.
    - 지시 데이터 대신 **선호도 데이터를 사용**하는 이점: **데이터가 표현하는 뉘앙스로 부터 모델이 사람의 기호에 맞는 출력을 생성하는 방법을 학습할 수 있음**.


In [ ]:
# httpx 패키지의 proxies 매개변수 오류를 피하기 위해
# https://community.openai.com/t/error-with-openai-1-56-0-client-init-got-an-unexpected-keyword-argument-proxies/
# !pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
# 아래 명령어 실행시 gradio와 pydantic 버전 오류 무시하기
!pip install openai httpx --upgrade

- **클로즈드 모델 사용하는 방법** : **API사용**
    1. **계정 생성** : https://openai.com/ko-KR/
    2. **API key 생성** :  https://platform.openai.com/api-keys
    3. **API key를 사용해 클라이언트(client) 만든다**.
    - **사용량(Usage) 확인** : https://platform.openai.com/usage
    - **가이드 확인**: https://platform.openai.com/docs/guides/rate-limits/retrying-with-exponential-backoff
    

In [ ]:
import openai
from tqdm import tqdm

# 클라이언트를 만듭니다.
client = openai.OpenAI(api_key="YOUR_KEY_HERE") # YOUR_KEY_HERE

- **chatgpt_generation 함수 만들기**
    - 프롬프트, 문서, 모델을 입력받아 텍스트 생성한다.

In [ ]:
def chatgpt_generation(prompt, document, model="gpt-4o-mini"):
    """프롬프트와 문서를 입력받아 출력을 생성합니다."""
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            },
        {
            "role": "user",
            "content":   prompt.replace("[DOCUMENT]", document)
            }
    ]
    chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      temperature=0
    )
    return chat_completion.choices[0].message.content

- **모델에게 요청하기 위한 템플릿 만들기**

In [ ]:
# 프롬프트 템플릿을 정의합니다.
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

- 💡[**주의**] 다음 단계는 전체 테스트 데이터로 오픈AI 모델 중 하나를 실행하는 것입니다. 하지만 전체 테스트 데이터는 1,066개이므로 충분한 크레딧이 있을 때만 실행하세요.

In [ ]:
# 무료 크레딧을 아끼고 싶다면 이 코드를 건너 뛰세요.
predictions = [chatgpt_generation(prompt, doc) for doc in tqdm(data["test"]["text"])]

In [ ]:
# 예측을 저장합니다.
y_pred = [int(pred) for pred in predictions]

# 성능을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)



---



## **[참고] ChatGTP 훈련 방법**

- 참고 : https://openai.com/ko-KR/index/chatgpt/
- **인간의 피드백을 활용한 강화학습**(Reinforcement Learning from Human Feedback, RLHF)의 3가지 핵심 단계
- ChatGPT와 같은 최신 대화형 AI 모델을 만드는 데 결정적인 역할을 한 기술
- **ChatGPT 훈련단계 Diagram**

![ChatGPT Diagram](https://images.ctfassets.net/kftzwdyauwt9/40in10B8KtAGrQvwRv5cop/8241bb17c283dced48ea034a41d7464a/chatgpt_diagram_light.png?w=1920&q=80&fm=webp)


### **1단계: 지도 미세조정 (SFT) - 모범 답안 학습하기**
이 단계는 AI에게 **좋은 답변이란 이런 것이다**라는 모범 사례를 직접 가르치는 과정입니다.

- **① 질문 (Prompt)**: "6살 아이에게 강화학습을 설명해줘"와 같은 지시문(프롬프트)을 준비합니다.

- **② 인간의 답변 작성** (Human Demonstration): 전문적인 데이터 라벨러가 이 질문에 대해 가장 이상적인, 고품질의 답변("간식과 벌을 주면서 가르치는 거야")을 직접 작성합니다.

- **③ 지도 미세조정** (Supervised Fine-Tuning, SFT): 이 '질문-모범 답변' 쌍을 거대한 언어 모델(LLM)에게 학습시킵니다. 모델은 이 과정을 통해 인간의 지시를 따르는 방법과 좋은 답변의 스타일 및 형식을 배우게 됩니다. 마치 학생에게 모범 답안지를 주고 그대로 따라 써보며 공부하게 하는 것과 같습니다.

결론적으로 1단계는, 기본적인 언어 능력만 있던 AI가 인간의 지시를 잘 따르는 '기초 소양'을 갖추게 하는 과정입니다.



### **2단계: 보상 모델 학습 (RM) - 좋은 답변을 알아보는 심사위원 만들기**
1단계만으로는 AI가 생성하는 다양한 답변 중 어떤 것이 더 나은지 판단할 수 없습니다. 그래서 이 단계에서는 **좋은 답변을 구별하는 눈을 가진 심사위원 AI**를 따로 만듭니다.

- **① 여러 답변 생성**: 1단계에서 SFT를 거친 모델에게 동일한 질문("6살 아이에게 강화학습을 설명해줘")을 주고, 여러 가지 다른 답변(A, B, C, D)을 생성하게 합니다.

- **② 인간의 순위 평가** (Human Ranking): 인간 라벨러가 AI가 생성한 여러 답변을 읽어보고, 가장 좋은 답변부터 가장 나쁜 답변까지 순위를 매깁니다. 이미지에서는 D > C > A > B 순서로 평가했습니다.

- **③ 보상 모델 학습** (Reward Model, RM): 이 '순위 데이터'를 가지고 별도의 AI 모델을 학습시킵니다. 이 모델이 바로 **보상 모델**(RM)입니다. RM은 답변을 직접 생성하지 않고, 대신 어떤 답변이 들어왔을 때 인간이 매긴 순위와 유사하게 '보상 점수'를 예측하도록 훈련됩니다. 즉, D 답변에는 높은 점수를, B 답변에는 낮은 점수를 주도록 학습하는 것입니다.

결론적으로 2단계는, 어떤 답변이 인간의 선호도에 더 부합하는지를 점수로 평가할 수 있는 AI 심사위원을 만드는 과정입니다.



### **3단계: 강화학습을 통한 미세조정 (PPO) - 심사위원에게 칭찬받으며 실력 키우기**
마지막 단계에서는 1단계에서 만든 '학생 AI'를 2단계에서 만든 '심사위원 AI'를 이용해 더욱 똑똑하게 만드는 본격적인 강화학습 과정입니다.

- **① 새로운 질문과 답변 생성**: 1단계의 SFT 모델(이제부터는 '정책(Policy)'이라고 부릅니다)에게 새로운 질문("수달에 대한 이야기를 써줘")을 던집니다. 모델은 학습한 대로 답변("옛날 옛적에...")을 생성합니다.

- **② 보상 모델의 평가**: 생성된 답변을 2단계에서 만든 '심사위원'인 보상 모델(RM)에게 보여줍니다. RM은 이 답변이 얼마나 좋은지 평가하여 **보상 점수**(rk)를 매깁니다.

- **③ 강화학습 알고리즘(PPO) 적용**: 이 보상 점수를 이용해 '학생 AI'를 업데이트합니다.

    - **높은 점수를 받으면** (칭찬): "아, 이런 식으로 답변하니까 좋아하는구나!"라고 배우며, 앞으로 그런 스타일의 답변을 더 잘 생성하도록 가중치를 조정합니다.

    - **낮은 점수를 받으면** (지적): "이런 답변은 별로인가 보네."라고 배우며, 해당 유형의 답변은 덜 생성하도록 가중치를 조정합니다.

- **④ 반복**: 이 과정을 수없이 반복하며, '학생 AI'는 어떻게 하면 '심사위원 AI'로부터 더 높은 보상 점수를 받을 수 있는지 스스로 터득하며 성능을 점차 향상시킵니다.

결론적으로 3단계는, AI가 스스로 답변을 생성하고, 인간의 선호도를 학습한 보상 모델로부터 피드백을 받으며 답변의 품질을 스스로 개선해나가는 핵심 과정입니다. 이 과정을 통해 AI는 단순히 지식을 나열하는 것을 넘어, 훨씬 더 자연스럽고 유용하며 안전한 답변을 생성할 수 있게 됩니다.